<a href="https://colab.research.google.com/github/iamjems/Enhancement-of-Surveillance-Cameras-Accident-Detection-with-Alert-System/blob/main/Partial_Accident_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/test folder'

In [ ]:
%pwd

In [ ]:
!git  clone https://github.com/iamjems/Enhancement-of-Surveillance-Cameras-Accident-Detection-with-Alert-System.git



**CODES**

In [ ]:
from ctypes import *
import math
import random
import os
import cv2
import numpy as np
import time
import darknet
from itertools import combinations
import pafy
import youtube_dl
#import image_email_car   # Uncomment for alert to email

def is_close(p1, p2):
    """
    #================================================================
    # Purpose : Calculate Euclidean Distance between two points
    #================================================================    
    :param:
    p1, p2 = two points for calculating Euclidean Distance
    :return:
    dst = Euclidean Distance between two 2d points
    """
    dst = math.sqrt(p1**2 + p2**2)
    #=================================================================#
    return dst 


def convertBack(x, y, w, h): 
    #================================================================
    # Purpose : Converts center coordinates to rectangle coordinates
    #================================================================  
    """
    :param:
    x, y = midpoint of bbox
    w, h = width, height of the bbox
    
    :return:
    xmin, ymin, xmax, ymax
    """
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax

#alert_var = 0      # Uncomment for alert to email       # makes sure that alert (Sending an E-mail) is generated only once
